# Calculating Impact with Different Exposures

This notebook goes through the calculation of the impact of tropical cyclones on New Zealand for two exposure types, `gdp` and `norm` (normalized to 1); these are set with `fin_mode`. This notebook shows that using `gdp` to calculate a fractional impact is the same as using `norm`.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from climada.entity import LitPop

from climada.entity.entity_def import Entity
from climada.entity import ImpactFuncSet, ImpactFunc
from climada.engine import Impact
from climada.engine import ImpactCalc

from climada.hazard import TCTracks, TropCyclone, Centroids
from climada.entity import ImpfTropCyclone

/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/dask/dataframe/_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 12.0.1. Please consider upgrading.
  warnings.warn(


### Define Hazard (Tropical Cyclone)

In [3]:
# Define exposure here only to get the centroid locations
# NZ with resolution 10km
exp_nz = LitPop.from_countries(countries=['NZL'],fin_mode='norm',res_arcsec=300)

# Define the centroids from the exposures position
lat = exp_nz.gdf['latitude'].values
lon = exp_nz.gdf['longitude'].values
centrs = Centroids.from_lat_lon(lat, lon)
centrs.check()

2024-08-03 13:42:20,304 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,325 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,338 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,351 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,379 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,392 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:20,407 - climada.entity.exposures.litpop.gpw_populati

In [10]:
# Load historical tropical cyclone tracks from ibtracs over the North Atlantic basin between 2010-2012
ibtracks_nz = TCTracks.from_ibtracs_netcdf(provider='wellington', basin='SP', year_range=(2010, 2023), correct_pres=True)
ibtracks_nz.equal_timestep(time_step_h=0.1)
print('num tracks hist:', ibtracks_nz.size)

# Using the tracks, compute the windspeed at the location of the centroids
haz_tc = TropCyclone.from_tracks(ibtracks_nz, centroids=centrs)
haz_tc.check()

2024-08-03 13:46:03,628 - climada.hazard.tc_tracks - WARNING - `correct_pres` is deprecated. Use `estimate_missing` instead.
2024-08-03 13:46:03,800 - climada.hazard.tc_tracks - WARNING - The cached IBTrACS data set dates from 2023-06-07 23:07:38 (older than 180 days). Very likely, a more recent version is available. Consider manually removing the file /home/ming/climada/data/IBTrACS.ALL.v04r00.nc and re-running this function, which will download the most recent version of the IBTrACS data set from the official URL.
2024-08-03 13:46:04,648 - climada.hazard.tc_tracks - WARNING - 94 storm events are discarded because no valid wind/pressure values have been found: 2010020S14147, 2010022S12160, 2010027S14184, 2010038S08194, 2010051S06195, ...


/home/ming/mambaforge/envs/climada_stable/lib/python3.9/site-packages/climada/hazard/tc_tracks.py:614: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if ibtracs_ds.dims['storm'] == 0:


num tracks hist: 47


### Impact function

In [5]:
# impact function TC
impf_tc = ImpfTropCyclone.from_emanuel_usa()

# add the impact function to an Impact function set
impf_set = ImpactFuncSet([impf_tc])

# Get the hazard type and hazard id
[haz_type] = impf_set.get_hazard_types()
[haz_id] = impf_set.get_ids()[haz_type]

### Exposure with gdp

In [6]:
# NZ with resolution 10km
exp_nz = LitPop.from_countries(countries=['NZL'],fin_mode='gdp',res_arcsec=300)

# Exposures: rename column and assign id
exp_nz.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_nz.gdf['impf_' + haz_type] = haz_id
exp_nz.check()

2024-08-03 13:42:33,002 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,026 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,041 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,056 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,087 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,102 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:33,119 - climada.entity.exposures.litpop.gpw_populati

In [7]:
# Compute impact
imp_tc = ImpactCalc(exp_nz, impf_set, haz_tc).impact()  # Do not save the results geographically resolved (only aggregate values)

# aai_agg is the Aggregated average annual impact
print(f"Aggregated average annual impact: {imp_tc.aai_agg}")

print(f"total_value: {imp_tc.tot_value,0}")

round(imp_tc.aai_agg,0)/round(imp_tc.tot_value,0)

Aggregated average annual impact: 136875.04685497616
2024-08-03 13:42:34,376 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
total_value: (211846555690.73596, 0)
2024-08-03 13:42:34,376 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold


6.461044389111819e-07

### Exposure with norm

In [8]:
# NZ with resolution 10km
exp_nz = LitPop.from_countries(countries=['NZL'],fin_mode='norm',res_arcsec=300)

# Exposures: rename column and assign id
exp_nz.gdf.rename(columns={"impf_": "impf_" + haz_type}, inplace=True)
exp_nz.gdf['impf_' + haz_type] = haz_id
exp_nz.check()

2024-08-03 13:42:36,671 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,692 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,706 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,720 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,748 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,761 - climada.entity.exposures.litpop.gpw_population - WARNING - Reference year: 2018. Using nearest available year for GPW data: 2020
2024-08-03 13:42:36,777 - climada.entity.exposures.litpop.gpw_populati

In [9]:
# Compute impact
imp_tc = ImpactCalc(exp_nz, impf_set, haz_tc).impact()  # Do not save the results geographically resolved (only aggregate values)

# aai_agg is the Aggregated average annual impact
print(f"Aggregated average annual impact: {imp_tc.aai_agg}") 

Aggregated average annual impact: 6.4610466008611e-07
